# Loading the Dataset

In [10]:
from datasets import load_dataset
import math
import ast

import pandas as pd

from tqdm import tqdm

tqdm.pandas(desc='bar')

/home/vaburban/efs/home/vaburban/occams_venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Running on only training data for now
cnn_dataset = load_dataset("cnn_dailymail", '3.0.0', split='train')
print(len(cnn_dataset))
cnn_dataset

Found cached dataset cnn_dailymail (/home/vaburban/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)


287113


Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 287113
})

In [ ]:
# Entire Dataframe
# cnn_summaries = pd.DataFrame(cnn_dataset).rename(columns={"article": "text", "highlights": "human_summary"})

# SAMPLING A SUBSET FOR EXPERIMENT
cnn_summaries = pd.DataFrame(cnn_dataset).rename(columns={"article": "text", "highlights": "human_summary"}).sample(150)

In [ ]:
avg_len = sum(cnn_summaries['human_summary'].str.len())/len(cnn_summaries)
char_budget = math.floor(avg_len) + 50
sentence_budget = math.floor(char_budget/50)
print(f"Character Budget: {char_budget}")
print(f"Sentence Budget: {sentence_budget}")

# OPTIONAL: Summarize using T5

This would be nice since we won't need to use the OpenAI API. However, Ani and John let me know this was trained on XSum for the summarization task and therefore will only yield single sentence summaries. This isn't great, so we will still want to bring in ChatGPT to evaluate.

In [8]:
# Had issues installing this bc of the venv, use 'source occams_venv/bin/activate' to get the terminal to work
# also had to install:
#   - sentencepiece
#   - pytorch
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

text = cnn_summaries.iloc[0]['occams_summ']

input_text = f"Tell me the main important sentences in this article: {text}"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids, max_length=300)
print(tokenizer.decode(outputs[0]))

<pad> Android chief Andy Rubin takes to the blogs to combat recent reports of Google clamping down on Android's openness.</s>


In [9]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")

text = cnn_summaries.iloc[1]['occams_summ']

input_text = f"summarize: {text}"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids, max_length=300)
print(tokenizer.decode(outputs[0]))

/home/vaburban/efs/home/vaburban/occams_venv/lib/python3.11/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


<pad> the PGA Tour has voted to change the rules on putting. the changes are controversial and will affect the world of golf. the PGA Tour runs the american circuit.</s>


In [10]:
# For fun, let's do a T5 summary on top of the hybrid summaries.
def t5_summarize(text):
    input_text = f"summarize: {text}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids
    outputs = model.generate(input_ids, max_length=300)
    return tokenizer.decode(outputs[0])


In [11]:
%%time
cnn_summaries["t5_hybrid_summ"] = cnn_summaries.occams_summ.progress_apply(t5_summarize)

bar: 100%|██████████| 100/100 [02:59<00:00,  1.80s/it]

CPU times: user 2min 59s, sys: 86.2 ms, total: 2min 59s
Wall time: 2min 59s


In [12]:
print(cnn_summaries.iloc[0]['human_summary'])
print()
print(cnn_summaries.iloc[0]['t5_hybrid_summ'])
print()
print(cnn_summaries.iloc[0]['occams_summ'])

Google has championed its platform as the open alternative to Apple's closed iOS system .
Honeycomb is Android's first tablet-optimized software release .
Rubin emphatically denied other rumors of ARM-chipset standardization in the platform .

<pad> Android chief rubin takes to the blogs to combat recent reports of Google clamping down on Android's openness. the android OS is based on a version of the Linux OS.</s>

(WIRED) -- Android chief Andy Rubin took to the blogs Wednesday evening to combat recent reports of Google clamping down on Android's openness. As soon as this work is completed, we'll publish the code. The Android OS is based on a version of the Linux OS, which has been an open source, collaborative platform since its release decades ago.


# Abstractive Summarization using GPT 3.5,

In [19]:
# code credit Jacob Baxter
import requests
with open('../artifacts/gpt_key.txt') as f:
    key_file = f.readlines()
KEY = key_file[0]
def gpt_summarize(text, prompt):
    url = "https://apiproxy.ncsu-las.net/APIGateway/openai/v1/chat/completions"

    headers = {
        "Content-Type": "application/json",
        "LAS-API-Token": KEY
    }

    data = {
      "model": "gpt-3.5-turbo",
      "messages": [{"role": "user", "content": prompt.format(text=text)}],
      "temperature": 0.7
    }

    response = requests.post(url, headers=headers, data=json.dumps(data))
    try:
        summ = response.json()["choices"][0]["message"]["content"]
    except:
        print("ERROR")
        summ = "No summary possible."
    return summ

In [12]:
%%time

# Prompt adapted from Extractive Summarization via ChatGPT for Faithful Summary Generation (Zhang et al)
abstractive_prompt = """You are an abstractive summarizer that follows the output pattern. Please write a summary for the following text in {char_budget} characters or less.

Text:
{text}

Summary:
""".format(char_budget=char_budget, text='{text}')

cnn_summaries['gpt_a_summ'] = cnn_summaries.text.progress_apply(lambda x: gpt_summarize(x, abstractive_prompt))

bar:  15%|█▌        | 23/150 [02:24<26:18, 12.43s/it]

ERROR


bar:  26%|██▌       | 39/150 [04:14<24:04, 13.02s/it]

ERROR


bar:  30%|███       | 45/150 [05:09<24:03, 13.75s/it]

ERROR


bar: 100%|██████████| 150/150 [14:08<00:00,  5.65s/it]

CPU times: user 9.98 s, sys: 90.8 ms, total: 10.1 s
Wall time: 14min 8s


In [13]:
print(len(cnn_summaries[cnn_summaries['gpt_a_summ']=='No summary possible.']))
cnn_summaries = cnn_summaries[cnn_summaries['gpt_a_summ']!='No summary possible.']

3


In [14]:
cnn_summaries.head()

,text,human_summary,id,occams_summ,gpt_a_summ
86826,"James Gandolfini, best known for his role as a...","Edie Falco, who played Tony Soprano's wife, is...",71997309bf0360f53411bd6af5d1fa95351bb117,"James Gandolfini, best known for his role as a...","Actor James Gandolfini, best known for his rol..."
110313,Geely pledges to retain group's 107 staff and ...,Coventry-based Manganese Bronze bought by Chin...,52e9acc58aaba7d5e3f32127ef458229bcc6f481,Geely pledges to retain group's 107 staff and ...,Chinese private car maker Geely has acquired C...
260717,Sachin Tendulkar has revealed that he missed t...,Sachin Tendulkar missed India's 2011 World Cup...,a1a1d975d616c427e23379a2f2d0ae5206e39f72,Sachin Tendulkar has revealed that he missed t...,"Sachin Tendulkar, the retired Indian cricketer..."
43256,(CNN) -- Neanderthals likely died off about 30...,"Archaeologists uncover 50,000-year-old special...",74d7298a8b989a5a74a131c60711d5a05a423a09,(CNN) -- Neanderthals likely died off about 30...,Neanderthals may have influenced later humans ...
226988,From the Suffragettes and Marilyn Monroe to Ri...,"The colour red is associated with warmth, posi...",f6f46082d0e1b8550ac33b3211692c0f01ea5212,From the Suffragettes and Marilyn Monroe to Ri...,Red lipstick has been used by women for decade...


In [15]:
cnn_summaries.to_parquet('datasets/cnn_experiment.parquet')

# Extractive Summarization Using GPT 3.5

In [19]:
%%time

# Prompt adapted from Extractive Summarization via ChatGPT for Faithful Summary Generation (Zhang et al)
extractive_prompt = """Please identify the {sentence_budget} most important sentences from the text, and present them in a Python list.

Text:
{text}

Output:
""".format(sentence_budget=sentence_budget, text='{text}')


cnn_summaries['gpt_e_summ'] = cnn_summaries.text.progress_apply(lambda x: gpt_summarize(x, extractive_prompt))

bar:  75%|███████▍  | 110/147 [14:11<08:54, 14.44s/it]

ERROR


bar:  78%|███████▊  | 115/147 [14:46<04:28,  8.38s/it]

ERROR


bar:  93%|█████████▎| 137/147 [18:01<02:18, 13.87s/it]

ERROR


bar: 100%|██████████| 147/147 [19:14<00:00,  7.86s/it]

CPU times: user 9.74 s, sys: 109 ms, total: 9.85 s
Wall time: 19min 14s


In [20]:
print(len(cnn_summaries[cnn_summaries['gpt_e_summ']=='No summary possible.']))
cnn_summaries = cnn_summaries[cnn_summaries['gpt_e_summ']!='No summary possible.']

3


In [21]:
cnn_summaries_copy = cnn_summaries.copy()

In [23]:
import ast
def parse_list(x):
    try:
        new_str = " ".join(ast.literal_eval(x))
        return new_str
    except:
        print('ERROR')
        print(x)
        new_str = 'ERROR'
        return new_str
    finally:
        return new_str
    
cnn_summaries['gpt_e_summ'] = cnn_summaries.gpt_e_summ.progress_apply(lambda x: parse_list(x))
cnn_summaries[cnn_summaries['gpt_e_summ']!='ERROR']

bar: 100%|██████████| 144/144 [00:00<00:00, 35708.87it/s]

ERROR
["Queen Elizabeth and the royal family cost British taxpayers an average of 66 pence ($1.32) per person last year, Buckingham Palace announced Friday in its annual report of royal finances.",
"The total cost of the queen and royal family was 40 million pounds ($80 million) in the past fiscal year, an increase of 2 percent from the year before, according to the Royal Public Finances report.",
"The reduction in the amount of head of state expenditure in real terms reflects the continuous attention the royal household pays to obtaining the best value for money in all areas of expenditure," said Alan Reid, whose official title is 'keeper of the privy purse.'",
"Reid warned that the money is not enough to deal with a backlog of maintenance work.",
"The most expensive trip was the queen's six-day state visit to the United States, which cost a total of 414,042 pounds (about $828,000).",
" 'This report is provided every year to show transparency in the royal accounts,' said CNN royal wat

,text,human_summary,id,occams_summ,gpt_a_summ,gpt_e_summ
86826,"James Gandolfini, best known for his role as a...","Edie Falco, who played Tony Soprano's wife, is...",71997309bf0360f53411bd6af5d1fa95351bb117,"James Gandolfini, best known for his role as a...","Actor James Gandolfini, best known for his rol...","James Gandolfini, best known for his role as a..."
110313,Geely pledges to retain group's 107 staff and ...,Coventry-based Manganese Bronze bought by Chin...,52e9acc58aaba7d5e3f32127ef458229bcc6f481,Geely pledges to retain group's 107 staff and ...,Chinese private car maker Geely has acquired C...,Geely pledged to retain the group's 107 staff ...
260717,Sachin Tendulkar has revealed that he missed t...,Sachin Tendulkar missed India's 2011 World Cup...,a1a1d975d616c427e23379a2f2d0ae5206e39f72,Sachin Tendulkar has revealed that he missed t...,"Sachin Tendulkar, the retired Indian cricketer...",Sachin Tendulkar has revealed that he missed t...
43256,(CNN) -- Neanderthals likely died off about 30...,"Archaeologists uncover 50,000-year-old special...",74d7298a8b989a5a74a131c60711d5a05a423a09,(CNN) -- Neanderthals likely died off about 30...,Neanderthals may have influenced later humans ...,Neanderthals were making specialized bone tool...
226988,From the Suffragettes and Marilyn Monroe to Ri...,"The colour red is associated with warmth, posi...",f6f46082d0e1b8550ac33b3211692c0f01ea5212,From the Suffragettes and Marilyn Monroe to Ri...,Red lipstick has been used by women for decade...,From the Suffragettes and Marilyn Monroe to Ri...
...,...,...,...,...,...,...
169751,By . Chloe Lambert . Peter Cross experienced p...,"Peter Cross, 61, spend time in hospital recove...",b84bc1465468e34857c8744c0fba7a331c714797,his surgery at University College London Hospi...,"Up to 50,000 people in the UK suffer from post...","Every year up to 50,000 people in Britain suff..."
83256,As the hours count down to $85 billion in gove...,Amtrak says no changes or cutbacks in schedule...,7ebfd63c63314ab632c37e29edbcc8fb00a90176,As the hours count down to $85 billion in gove...,Despite the impending government-wide spending...,Officials of the nation's only long-haul passe...
55507,"Donetsk, Ukraine (CNN) -- The battle for contr...",Civilians caught in a crossfire between Ukrain...,5c2a93464009e928258d2588789c6ff8ced60fb2,"Donetsk, Ukraine (CNN) -- The battle for contr...",The battle for control of the eastern Ukrainia...,The battle for control of the eastern Ukrainia...
98340,By . Daily Mail Reporter . UPDATED: . 10:59 ES...,U.S trials halted after twice as many patients...,bc86832fbef69988507351d5ef030bdbcb340b89,Artery-opening brain stents that are designed ...,Brain stents designed to prevent repeat stroke...,Artery-opening brain stents that are designed ...


In [25]:
cnn_summaries = cnn_summaries[cnn_summaries['gpt_e_summ']!='ERROR']

In [27]:
print(len(cnn_summaries))
cnn_summaries.head()

135


,text,human_summary,id,occams_summ,gpt_a_summ,gpt_e_summ
86826,"James Gandolfini, best known for his role as a...","Edie Falco, who played Tony Soprano's wife, is...",71997309bf0360f53411bd6af5d1fa95351bb117,"James Gandolfini, best known for his role as a...","Actor James Gandolfini, best known for his rol...","James Gandolfini, best known for his role as a..."
110313,Geely pledges to retain group's 107 staff and ...,Coventry-based Manganese Bronze bought by Chin...,52e9acc58aaba7d5e3f32127ef458229bcc6f481,Geely pledges to retain group's 107 staff and ...,Chinese private car maker Geely has acquired C...,Geely pledged to retain the group's 107 staff ...
260717,Sachin Tendulkar has revealed that he missed t...,Sachin Tendulkar missed India's 2011 World Cup...,a1a1d975d616c427e23379a2f2d0ae5206e39f72,Sachin Tendulkar has revealed that he missed t...,"Sachin Tendulkar, the retired Indian cricketer...",Sachin Tendulkar has revealed that he missed t...
43256,(CNN) -- Neanderthals likely died off about 30...,"Archaeologists uncover 50,000-year-old special...",74d7298a8b989a5a74a131c60711d5a05a423a09,(CNN) -- Neanderthals likely died off about 30...,Neanderthals may have influenced later humans ...,Neanderthals were making specialized bone tool...
226988,From the Suffragettes and Marilyn Monroe to Ri...,"The colour red is associated with warmth, posi...",f6f46082d0e1b8550ac33b3211692c0f01ea5212,From the Suffragettes and Marilyn Monroe to Ri...,Red lipstick has been used by women for decade...,From the Suffragettes and Marilyn Monroe to Ri...


In [28]:
cnn_summaries.to_parquet('datasets/cnn_experiment.parquet')

In [2]:
import pandas as pd
cnn_summaries = pd.read_parquet('datasets/cnn_experiment.parquet')

## OCCAMS Extractive Summaries

In [6]:
from occams.nlp import TermOrder
from occams.nlp import process_document
from occams.summarize import SummaryUnits, extract_summary
UNITS = SummaryUnits.CHARS

In [12]:
# avg_len = sum(cnn_summaries['gpt_e_summ'].str.len())/len(cnn_summaries)
# char_budget = cnn_summaries.gpt_e_summ.str.len().max()
char_budget = math.floor(cnn_summaries.gpt_e_summ.str.len().quantile(.75))

from nltk.tokenize import sent_tokenize

# Adapted from Jacob Baxter
def occams_summarize(document):
    return extract_summary([process_document(document, TermOrder.BIGRAMS)], budget=char_budget, units=UNITS).summary()
    

In [13]:
%%time 
# cnn_summaries["occams_summ"] = cnn_summaries.head(10).text.apply(occams_summarize)
cnn_summaries["occams_summ"] = cnn_summaries.text.progress_apply(occams_summarize)
cnn_summaries.head()

bar: 100%|██████████| 129/129 [00:01<00:00, 72.40it/s]

CPU times: user 1.77 s, sys: 11.8 ms, total: 1.78 s
Wall time: 1.78 s


,text,human_summary,id,occams_summ,gpt_a_summ,gpt_e_summ,occams_gpt_summ,gpt_gpt_summ
86826,"James Gandolfini, best known for his role as a...","Edie Falco, who played Tony Soprano's wife, is...",71997309bf0360f53411bd6af5d1fa95351bb117,"James Gandolfini, best known for his role as a...","Actor James Gandolfini, best known for his rol...","James Gandolfini, best known for his role as a...","James Gandolfini, famous for his role in ""The ...","James Gandolfini, best known for his role as T..."
110313,Geely pledges to retain group's 107 staff and ...,Coventry-based Manganese Bronze bought by Chin...,52e9acc58aaba7d5e3f32127ef458229bcc6f481,Geely pledges to retain group's 107 staff and ...,Chinese private car maker Geely has acquired C...,Geely pledged to retain the group's 107 staff ...,Geely has promised to keep the London Taxi Com...,Chinese automaker Geely has acquired London Ta...
260717,Sachin Tendulkar has revealed that he missed t...,Sachin Tendulkar missed India's 2011 World Cup...,a1a1d975d616c427e23379a2f2d0ae5206e39f72,Sachin Tendulkar has revealed that he missed t...,"Sachin Tendulkar, the retired Indian cricketer...",Sachin Tendulkar has revealed that he missed t...,Sachin Tendulkar missed India's victory in the...,"Sachin Tendulkar, the retired Indian cricketer..."
43256,(CNN) -- Neanderthals likely died off about 30...,"Archaeologists uncover 50,000-year-old special...",74d7298a8b989a5a74a131c60711d5a05a423a09,(CNN) -- Neanderthals likely died off about 30...,Neanderthals may have influenced later humans ...,Neanderthals were making specialized bone tool...,A new study suggests that Neanderthals may hav...,Neanderthals had advanced behavioral complexit...
226988,From the Suffragettes and Marilyn Monroe to Ri...,"The colour red is associated with warmth, posi...",f6f46082d0e1b8550ac33b3211692c0f01ea5212,From the Suffragettes and Marilyn Monroe to Ri...,Red lipstick has been used by women for decade...,From the Suffragettes and Marilyn Monroe to Ri...,Red lipstick has been used by women for decade...,Scientists at the University of Manchester fou...


In [16]:
cnn_summaries.to_parquet('datasets/cnn_experiment.parquet')

In [15]:
cnn_summaries.occams_summ.str.len().describe()


count     129.000000
mean      982.565891
std        34.280061
min       646.000000
25%       978.000000
50%       988.000000
75%      1000.000000
max      1004.000000
Name: occams_summ, dtype: float64

# Hybrid Summarization

## Extractive: OCCAMS Abstractive: GPT 3.5

In [20]:
%%time

# Note that we are using the SAME prompt for both OCCAMS and ChatGPT based extractive summaries
hybrid_prompt = """You are a  summarizer that follows the output pattern. Please summarize this text.

Text:
{text}

Summary:
""".format(text='{text}')

cnn_summaries['occams_gpt_summ'] = cnn_summaries.occams_summ.progress_apply(lambda x: gpt_summarize(x, hybrid_prompt))

bar:  49%|████▉     | 63/129 [04:43<13:12, 12.00s/it]

ERROR


bar:  50%|████▉     | 64/129 [04:48<10:43,  9.90s/it]

ERROR


bar:  79%|███████▉  | 102/129 [07:55<05:28, 12.17s/it]

ERROR


bar: 100%|██████████| 129/129 [09:53<00:00,  4.60s/it]

CPU times: user 7.67 s, sys: 83.7 ms, total: 7.75 s
Wall time: 9min 53s


In [21]:
print(len(cnn_summaries[cnn_summaries['occams_gpt_summ']=='No summary possible.']))
cnn_summaries = cnn_summaries[cnn_summaries['occams_gpt_summ']!='No summary possible.']

cnn_summaries.head()

3


,text,human_summary,id,occams_summ,gpt_a_summ,gpt_e_summ,occams_gpt_summ,gpt_gpt_summ
86826,"James Gandolfini, best known for his role as a...","Edie Falco, who played Tony Soprano's wife, is...",71997309bf0360f53411bd6af5d1fa95351bb117,"James Gandolfini, best known for his role as a...","Actor James Gandolfini, best known for his rol...","James Gandolfini, best known for his role as a...","James Gandolfini, best known for his role as T...","James Gandolfini, best known for his role as T..."
110313,Geely pledges to retain group's 107 staff and ...,Coventry-based Manganese Bronze bought by Chin...,52e9acc58aaba7d5e3f32127ef458229bcc6f481,Geely pledges to retain group's 107 staff and ...,Chinese private car maker Geely has acquired C...,Geely pledged to retain the group's 107 staff ...,Chinese company Geely has saved the London Tax...,Chinese automaker Geely has acquired London Ta...
260717,Sachin Tendulkar has revealed that he missed t...,Sachin Tendulkar missed India's 2011 World Cup...,a1a1d975d616c427e23379a2f2d0ae5206e39f72,Sachin Tendulkar has revealed that he missed t...,"Sachin Tendulkar, the retired Indian cricketer...",Sachin Tendulkar has revealed that he missed t...,"Sachin Tendulkar, former Indian cricketer, adm...","Sachin Tendulkar, the retired Indian cricketer..."
43256,(CNN) -- Neanderthals likely died off about 30...,"Archaeologists uncover 50,000-year-old special...",74d7298a8b989a5a74a131c60711d5a05a423a09,(CNN) -- Neanderthals likely died off about 30...,Neanderthals may have influenced later humans ...,Neanderthals were making specialized bone tool...,A new study suggests that Neanderthals may hav...,Neanderthals had advanced behavioral complexit...
226988,From the Suffragettes and Marilyn Monroe to Ri...,"The colour red is associated with warmth, posi...",f6f46082d0e1b8550ac33b3211692c0f01ea5212,From the Suffragettes and Marilyn Monroe to Ri...,Red lipstick has been used by women for decade...,From the Suffragettes and Marilyn Monroe to Ri...,Red lipstick has been used by women for decade...,Scientists at the University of Manchester fou...


## Extractive: GPT 3.5 Abstractive: GPT 3.5

In [32]:
%%time

# Note that we are using the SAME prompt for both OCCAMS and ChatGPT based extractive summaries
hybrid_prompt = """You are a  summarizer that follows the output pattern. Please summarize this text.

Text:
{text}

Summary:
""".format(sentence_budget=sentence_budget, text='{text}')

cnn_summaries['gpt_gpt_summ'] = cnn_summaries.gpt_e_summ.progress_apply(lambda x: gpt_summarize(x, hybrid_prompt))

bar:  18%|█▊        | 24/132 [01:55<20:42, 11.50s/it]

ERROR


bar:  48%|████▊     | 63/132 [04:51<12:57, 11.27s/it]

ERROR


bar:  48%|████▊     | 64/132 [04:55<10:27,  9.22s/it]

ERROR


bar: 100%|██████████| 132/132 [09:40<00:00,  4.40s/it]

CPU times: user 8.7 s, sys: 66.4 ms, total: 8.77 s
Wall time: 9min 40s


In [34]:
print(len(cnn_summaries[cnn_summaries['gpt_gpt_summ']=='No summary possible.']))
cnn_summaries = cnn_summaries[cnn_summaries['gpt_gpt_summ']!='No summary possible.']
cnn_summaries.head()

0


,text,human_summary,id,occams_summ,gpt_a_summ,gpt_e_summ,occams_gpt_summ,gpt_gpt_summ
86826,"James Gandolfini, best known for his role as a...","Edie Falco, who played Tony Soprano's wife, is...",71997309bf0360f53411bd6af5d1fa95351bb117,"James Gandolfini, best known for his role as a...","Actor James Gandolfini, best known for his rol...","James Gandolfini, best known for his role as a...","James Gandolfini, famous for his role in ""The ...","James Gandolfini, best known for his role as T..."
110313,Geely pledges to retain group's 107 staff and ...,Coventry-based Manganese Bronze bought by Chin...,52e9acc58aaba7d5e3f32127ef458229bcc6f481,Geely pledges to retain group's 107 staff and ...,Chinese private car maker Geely has acquired C...,Geely pledged to retain the group's 107 staff ...,Geely has promised to keep the London Taxi Com...,Chinese automaker Geely has acquired London Ta...
260717,Sachin Tendulkar has revealed that he missed t...,Sachin Tendulkar missed India's 2011 World Cup...,a1a1d975d616c427e23379a2f2d0ae5206e39f72,Sachin Tendulkar has revealed that he missed t...,"Sachin Tendulkar, the retired Indian cricketer...",Sachin Tendulkar has revealed that he missed t...,Sachin Tendulkar missed India's victory in the...,"Sachin Tendulkar, the retired Indian cricketer..."
43256,(CNN) -- Neanderthals likely died off about 30...,"Archaeologists uncover 50,000-year-old special...",74d7298a8b989a5a74a131c60711d5a05a423a09,(CNN) -- Neanderthals likely died off about 30...,Neanderthals may have influenced later humans ...,Neanderthals were making specialized bone tool...,A new study suggests that Neanderthals may hav...,Neanderthals had advanced behavioral complexit...
226988,From the Suffragettes and Marilyn Monroe to Ri...,"The colour red is associated with warmth, posi...",f6f46082d0e1b8550ac33b3211692c0f01ea5212,From the Suffragettes and Marilyn Monroe to Ri...,Red lipstick has been used by women for decade...,From the Suffragettes and Marilyn Monroe to Ri...,Red lipstick has been used by women for decade...,Scientists at the University of Manchester fou...


In [35]:
cnn_summaries.to_parquet('datasets/cnn_experiment.parquet')

# IS GPT REALLY EXTRACTIVE?

In [73]:
%%time
import json
import ast
experiment_df = cnn_summaries[['text']]

extractive_prompt = """Please identify the {sentence_budget} most important sentences from the text, and present them in a Python list.

Text:
{text}

Output:
""".format(sentence_budget=sentence_budget, text='{text}')


experiment_df['experiment'] = experiment_df.text.progress_apply(lambda x: gpt_summarize(x, extractive_prompt))


bar: 100%|██████████| 100/100 [13:51<00:00,  8.32s/it]

CPU times: user 5.9 s, sys: 85.1 ms, total: 5.99 s
Wall time: 13min 51s



<timed exec>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [74]:
# gpt_summarize(experiment_df.text.iloc[0], extractive_prompt)
" ".join(ast.literal_eval(gpt_summarize(experiment_df.text.iloc[0], extractive_prompt)))

def parse_list(x):
    try:
        new_str = " ".join(ast.literal_eval(x))
        return new_str
    except:
        print('ERROR')
        print(x)
        new_str = 'ERROR'
        return new_str
    finally:
        return new_str

experiment_df['paragraph'] = experiment_df.experiment.progress_apply(lambda x: parse_list(x))
experiment_df

bar: 100%|██████████| 100/100 [00:00<00:00, 38940.71it/s]

ERROR
["Fernando Alonso won't be saving Euskaltel Euskadi after all.", 
"But Alonso still wants a cycling team and might start one up from 'scratch.'", 
"If the Spanish outfit can't find another major sponsor soon, it won't be around for a 20th year.", 
"The negotiations between Euskaltel and the representatives of Fernando Alonso for the acquisition of the cycling team Euskaltel Euskadi on behalf of the Asturian driver, which began with an agreement in principle on August 31, have ended without an agreement," 
"We've tried it until the end but it's just been impossible to have a cycling team in 2014," said Alonso. 
"From tomorrow morning we are going to work on building, if need be from scratch, a team we can be proud of."]
ERROR
[
"Hurricane Barbara weakened to a tropical storm Wednesday after it crashed ashore along Mexico's southern Pacific coast.", 
"Barbara had sustained winds then of 75 mph, and was moving north-northeast at 9 mph.", 
"Winds weakened to 50 mph by Wednesday night


/tmp/ipykernel_6006/850572387.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  experiment_df['paragraph'] = experiment_df.experiment.progress_apply(lambda x: parse_list(x))


,text,experiment,paragraph
15159,(CNN) -- Seven-time world champion Michael Sch...,"[""Seven-time world champion Michael Schumacher...",Seven-time world champion Michael Schumacher h...
229936,"Notoriety: Richardson, pictured in 2007, was t...","[""Infamous London gang leader and rival to the...",Infamous London gang leader and rival to the K...
285097,"A team of diggers has started removing 2,500 t...","[\n""A team of diggers has started removing 2,5...","A team of diggers has started removing 2,500 t..."
61927,"Rear Adm. Anne Schuchat, MD, is assistant surg...","[""This week, the world has come together to fo...","This week, the world has come together to focu..."
226443,The stepfather of a woman held by police for t...,['The law must now be changed to protect prope...,The law must now be changed to protect propert...
...,...,...,...
86343,"Six months ago, the story began as a puzzling ...","[\n""Six months ago, the story began as a puzzl...","Six months ago, the story began as a puzzling ..."
196070,"Wayne Payne, 31, was travelling at 61mph when ...","[\n""Wayne Payne, 31, was travelling at 61mph w...","Wayne Payne, 31, was travelling at 61mph when ..."
8224,(CNN) -- Winter travelers trek thousands of mi...,"[\n""Scientists call the natural phenomenon aur...",Scientists call the natural phenomenon aurora ...
107185,"By . Larisa Brown . PUBLISHED: . 08:01 EST, 17...","[""'We used to have good fun when we were clean...",'We used to have good fun when we were cleanin...


In [76]:
errored_df = experiment_df[experiment_df['paragraph']== 'ERROR']
rewrite_prompt = """Please rewrite the following to be a valid Python list of strings:

{text}

""".format(text='{text}')
errored_df['rewrite'] = errored_df.experiment.progress_apply(lambda x: gpt_summarize(x, rewrite_prompt))
errored_df['paragraph'] = errored_df.rewrite.progress_apply(lambda x: parse_list(x))
errored_df

bar: 100%|██████████| 8/8 [00:56<00:00,  7.10s/it]
/tmp/ipykernel_6006/1337259432.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errored_df['rewrite'] = errored_df.experiment.progress_apply(lambda x: gpt_summarize(x, rewrite_prompt))
bar: 100%|██████████| 8/8 [00:00<00:00, 10061.30it/s]

ERROR
['Fernando Alonso won\'t be saving Euskaltel Euskadi after all.', 
"But Alonso still wants a cycling team and might start one up from 'scratch.'", 
"If the Spanish outfit can't find another major sponsor soon, it won't be around for a 20th year.", 
"The negotiations between Euskaltel and the representatives of Fernando Alonso for the acquisition of the cycling team Euskaltel Euskadi on behalf of the Asturian driver, which began with an agreement in principle on August 31, have ended without an agreement,", 
"We've tried it until the end but it's just been impossible to have a cycling team in 2014," said Alonso. 
"From tomorrow morning we are going to work on building, if need be from scratch, a team we can be proud of."]



/tmp/ipykernel_6006/1337259432.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errored_df['paragraph'] = errored_df.rewrite.progress_apply(lambda x: parse_list(x))


,text,experiment,paragraph,rewrite
49261,(CNN) -- Fernando Alonso won't be saving Euska...,"[""Fernando Alonso won't be saving Euskaltel Eu...",ERROR,['Fernando Alonso won\'t be saving Euskaltel E...
46775,(CNN) -- Hurricane Barbara weakened to a tropi...,"[\n""Hurricane Barbara weakened to a tropical s...",Hurricane Barbara weakened to a tropical storm...,"[\n""Hurricane Barbara weakened to a tropical s..."
112141,"By . Steve Nolan . PUBLISHED: . 07:33 EST, 7 F...","[""A workman suffered horrific burns to his arm...",A workman suffered horrific burns to his arm a...,"[""A workman suffered horrific burns to his arm..."
244889,The way millions of households are being rippe...,['Figures show customers are being charged at ...,Figures show customers are being charged at le...,"[""Figures show customers are being charged at ..."
87813,(CNN)Linda Kaoma is on the hunt in East Africa...,"[""We have a rich oral tradition and it's impor...",We have a rich oral tradition and it's importa...,['We have a rich oral tradition and it\'s impo...
41182,"Atlanta (CNN) -- Bishop Eddie Long, head of on...",No summary possible.,No summary possible.,"[""No summary possible.""]"
29875,(CNN) -- Germany's Tommy Haas upset Roger Fede...,"[\n""Germany's Tommy Haas upset Roger Federer i...",Germany's Tommy Haas upset Roger Federer in st...,"[\n""Germany's Tommy Haas upset Roger Federer i..."
36770,Beijing (CNN) -- Divining what is going in Chi...,['The 18th Congress of the 82-million-member C...,The 18th Congress of the 82-million-member Chi...,"[\n ""The 18th Congress of the 82-million-me..."


In [ ]:
['Fernando Alonso won\'t be saving Euskaltel Euskadi after all.', 
"But Alonso still wants a cycling team and might start one up from 'scratch.'", 
"If the Spanish outfit can't find another major sponsor soon, it won't be around for a 20th year.", 
"The negotiations between Euskaltel and the representatives of Fernando Alonso for the acquisition of the cycling team Euskaltel Euskadi on behalf of the Asturian driver, which began with an agreement in principle on August 31, have ended without an agreement,", 
"We've tried it until the end but it's just been impossible to have a cycling team in 2014," said Alonso. 
"From tomorrow morning we are going to work on building, if need be from scratch, a team we can be proud of."]


In [68]:
experiment_df = experiment_df[experiment_df['paragraph']!= 'ERROR']

In [71]:
from nltk.tokenize import sent_tokenize

experiment_df['tokenized_e'] = experiment_df['experiment'].apply(sent_tokenize)
experiment_df.head()

/tmp/ipykernel_6006/916994109.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  experiment_df['tokenized_e'] = experiment_df['experiment'].apply(sent_tokenize)


,text,experiment,paragraph,tokenized_e,num_different
15159,(CNN) -- Seven-time world champion Michael Sch...,"[""Seven-time world champion Michael Schumacher...",Seven-time world champion Michael Schumacher h...,"[[""Seven-time world champion Michael Schumache...",0.000000
229936,"Notoriety: Richardson, pictured in 2007, was t...",['Infamous London gang leader and rival to the...,Infamous London gang leader and rival to the K...,[['Infamous London gang leader and rival to th...,0.000000
285097,"A team of diggers has started removing 2,500 t...","[\n""A team of diggers has started removing 2,5...","A team of diggers has started removing 2,500 t...","[[\n""A team of diggers has started removing 2,...",0.166667
61927,"Rear Adm. Anne Schuchat, MD, is assistant surg...","[""This week, the world has come together to fo...","This week, the world has come together to focu...","[[""This week, the world has come together to f...",0.000000
226443,The stepfather of a woman held by police for t...,['The law must now be changed to protect prope...,The law must now be changed to protect propert...,[['The law must now be changed to protect prop...,0.071429


In [72]:
import string
  

def match_sent(sent1, sent2):
    return sent1.translate(str.maketrans('', '', string.punctuation)) in sent2.translate(str.maketrans('', '', string.punctuation))

def sentences_contained(row):
    tokens = row['tokenized_e']
    counter = 0
    for token in tokens:
        if not match_sent(token, row['text']) and len(token)>3:
            counter = counter + 1
    return counter / len(tokens)

experiment_df['num_different'] = experiment_df.apply(sentences_contained, axis=1)
experiment_df['num_different'].describe()

/tmp/ipykernel_6006/2247963862.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  experiment_df['num_different'] = experiment_df.apply(sentences_contained, axis=1)


count    91.000000
mean      0.687156
std       0.286919
min       0.000000
25%       0.651515
50%       0.833333
75%       0.833333
max       1.000000
Name: num_different, dtype: float64

In [24]:
output_t = [
"Seven-time world champion Michael Schumacher has told CNN that the Red Bull team should have already tied up the Formula One drivers' title -- and can only blame themselves for not having done so.",
"Fernando Alonso is in pole position to win his third crown, this time for Schumacher's former team Ferrari, at the 2010 season finale in Abu Dhabi this weekend.",
"The Austrian outfit has already clinched the constructors' title, but Schumacher claims the drivers' battle should not have gone to the final race -- hinting at the widespread reports of tension between Vettel and Webber.",
"I guess the main reason why the championship is still open is due to the fact that one particular team has made quite a few errors this year,\" he said.",
"Schumacher's return to F1 may not have been quite as successful as he would have hoped, but the German believes his past success has helped his younger compatriots.",
"Schumacher, who dominated motorsport's elite class with Ferrari before retiring in late 2006, said he deserved some credit for raising F1's popularity in his home country."
]

input_t = experiment_df.text.iloc[0]

for output in output_t:
    print(match_sent(output, input_t))

True
True
True
True
True
True


In [29]:
samp = experiment_df.experiment.iloc[0]
samp

'["Seven-time world champion Michael Schumacher has told CNN that the Red Bull team should have already tied up the Formula One drivers\' title -- and can only blame themselves for not having done so.",\n "The Spaniard has an eight-point lead from Mark Webber, with the Australian\'s Red Bull teammate Sebastian Vettel another seven points off the pace.",\n \'"I guess the main reason why the championship is still open is due to the fact that one particular team has made quite a few errors this year," he said.\',\n \'"And if you think out of 18 races they took 15 pole positions and still only being second and third in the drivers\\\' championship -- they want to win the championship, yes."\',\n "Schumacher\'s return to F1 may not have been quite as successful as he would have hoped, but the German believes his past success has helped his younger compatriots.",\n \'"Well the future is very clearly to make step by step the development that we really have initiated this year and take the fru

In [35]:
import ast
samp[2:-2]
" ".join(ast.literal_eval(samp))

'Seven-time world champion Michael Schumacher has told CNN that the Red Bull team should have already tied up the Formula One drivers\' title -- and can only blame themselves for not having done so. The Spaniard has an eight-point lead from Mark Webber, with the Australian\'s Red Bull teammate Sebastian Vettel another seven points off the pace. "I guess the main reason why the championship is still open is due to the fact that one particular team has made quite a few errors this year," he said. "And if you think out of 18 races they took 15 pole positions and still only being second and third in the drivers\' championship -- they want to win the championship, yes." Schumacher\'s return to F1 may not have been quite as successful as he would have hoped, but the German believes his past success has helped his younger compatriots. "Well the future is very clearly to make step by step the development that we really have initiated this year and take the fruits of it by starting [well] next 

In [ ]:
["Doctor Who" executive producer Stephen Moffat announced that actress Jenna-Louise Coleman will play Matt Smith's new companion following the departure of Karen Gillan and Arthur Darvill later this year., "Jenna is going to lead him on his merriest dance yet., Even by the Doctor's standards, this isn't your usual boy-meets-girl., The 25-year-old Coleman's credits include the British soap "Emmerdale," "Captain America," and the upcoming miniseries "Titanic.", Moffat revealed at a press conference that Gillan's Amy and Darvill's Rory would make their exit from the show in the fifth episode of the new season., "It's not often the Doctor meets someone who can talk even faster than he does, but it's about to happen," Moffat said in a statement.]



# Evaluating with ROUGE

In [5]:
from occams.rouge_eval import rouge_n_r
import numpy as np
from occams.nlp import TermOrder
from occams.nlp import process_document
from occams.summarize import SummaryUnits, extract_summary
UNITS = SummaryUnits.CHARS

char_budget = 350
def compute_rouge_scores(extracts, summaries, target_length, units=SummaryUnits.WORDS):
    rouges = []
    for occ_summary, model in zip(extracts, summaries):
        rouge = rouge_n_r(
            occ_summary, model, summary_length=target_length, units=units, max_n=4
            # occ_summary, model, summary_length=target_length, units=units
        )
        rouges.append(rouge[1:])
    rouges = np.array(rouges)
    return rouges


rouges = compute_rouge_scores(
    cnn_summaries['occams_summ'], cnn_summaries["human_summary"], char_budget, units=UNITS
)

print("occams")
print(np.mean(rouges, axis=0))

rouges = compute_rouge_scores(
    cnn_summaries['t5_hybrid_summ'], cnn_summaries["human_summary"], char_budget, units=UNITS
)

print("t5")
print(np.mean(rouges, axis=0))

rouges = compute_rouge_scores(
    cnn_summaries['gpt_a_summ'], cnn_summaries["human_summary"], char_budget, units=UNITS
)

print("gpt")
print(np.mean(rouges, axis=0))

occams
[0.38748881 0.13210813 0.06763282 0.04228074]
t5
[0.22088476 0.07240466 0.04137584 0.02880853]
gpt
[0.4677695  0.18611936 0.09552272 0.05594096]


In [11]:
df=pd.DataFrame(rouges)
df[df.columns[2:-1]].mean()

2    0.095523
dtype: float64

In [12]:
rouges

array([[0.52777778, 0.05714286, 0.        , 0.        ],
       [0.6       , 0.20408163, 0.08333333, 0.0212766 ],
       [0.53125   , 0.23809524, 0.08064516, 0.03278689],
       [0.38983051, 0.22413793, 0.15789474, 0.10714286],
       [0.38235294, 0.09090909, 0.0625    , 0.03225806],
       [0.46031746, 0.16129032, 0.06557377, 0.01666667],
       [0.25454545, 0.07407407, 0.01886792, 0.        ],
       [0.35714286, 0.14545455, 0.11111111, 0.09433962],
       [0.3220339 , 0.10344828, 0.03508772, 0.        ],
       [0.43103448, 0.15789474, 0.03571429, 0.        ],
       [0.70454545, 0.48837209, 0.28571429, 0.12195122],
       [0.42592593, 0.11320755, 0.01923077, 0.        ],
       [0.45      , 0.16949153, 0.0862069 , 0.03508772],
       [0.4375    , 0.21276596, 0.08695652, 0.04444444],
       [0.24      , 0.02040816, 0.        , 0.        ],
       [0.36585366, 0.075     , 0.02564103, 0.        ],
       [0.28333333, 0.03389831, 0.        , 0.        ],
       [0.27272727, 0.        ,

In [8]:
import evaluate
rouge = evaluate.load('rouge')

rouge = evaluate.load('rouge')
predictions = ["hello goodbye", "ankh morpork"]
references = ["goodbye", "general kenobi"]
results = rouge.compute(predictions=predictions,
                             references=references,
                             use_aggregator=False)
print(list(results.keys()))
print(results["rouge1"])


['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
[0.6666666666666666, 0.0]


In [10]:
from rouge import Rouge 
hypothesis = "the #### transcript is a written version of each day 's cnn student news program use this transcript to he    lp students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you     saw on cnn student news"
reference = "this page includes the show transcript use the transcript to help students with reading comprehension and     vocabulary at the bottom of the page , comment for a chance to be mentioned on cnn student news . you must be a teac    her or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests     students ' knowledge of even ts in the news"
rouge = Rouge()
scores = rouge.get_scores(hypothesis, reference)
scores

[{'rouge-1': {'f': 0.4786324739396596,
   'p': 0.6363636363636364,
   'r': 0.3835616438356164},
  'rouge-2': {'f': 0.2608695605353498,
   'p': 0.3488372093023256,
   'r': 0.20833333333333334},
  'rouge-l': {'f': 0.44705881864636676,
   'p': 0.5277777777777778,
   'r': 0.3877551020408163}}]

In [12]:
import rouge


def prepare_results(p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(metric, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)


for aggregator in ['Avg', 'Best', 'Individual']:
    print('Evaluation with {}'.format(aggregator))
    apply_avg = aggregator == 'Avg'
    apply_best = aggregator == 'Best'

    evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                           # max_n=4,
                           # limit_length=True,
                           length_limit=100,
                           length_limit_type='words',
                           apply_avg=apply_avg,
                           apply_best=apply_best,
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)


    hypothesis_1 = "King Norodom Sihanouk has declined requests to chair a summit of Cambodia 's top political leaders , saying the meeting would not bring any progress in deadlocked negotiations to form a government .\nGovernment and opposition parties have asked King Norodom Sihanouk to host a summit meeting after a series of post-election negotiations between the two opposition groups and Hun Sen 's party to form a new government failed .\nHun Sen 's ruling party narrowly won a majority in elections in July , but the opposition _ claiming widespread intimidation and fraud _ has denied Hun Sen the two-thirds vote in parliament required to approve the next government .\n"
    references_1 = ["Prospects were dim for resolution of the political crisis in Cambodia in October 1998.\nPrime Minister Hun Sen insisted that talks take place in Cambodia while opposition leaders Ranariddh and Sam Rainsy, fearing arrest at home, wanted them abroad.\nKing Sihanouk declined to chair talks in either place.\nA U.S. House resolution criticized Hun Sen's regime while the opposition tried to cut off his access to loans.\nBut in November the King announced a coalition government with Hun Sen heading the executive and Ranariddh leading the parliament.\nLeft out, Sam Rainsy sought the King's assurance of Hun Sen's promise of safety and freedom for all politicians.",
                    "Cambodian prime minister Hun Sen rejects demands of 2 opposition parties for talks in Beijing after failing to win a 2/3 majority in recent elections.\nSihanouk refuses to host talks in Beijing.\nOpposition parties ask the Asian Development Bank to stop loans to Hun Sen's government.\nCCP defends Hun Sen to the US Senate.\nFUNCINPEC refuses to share the presidency.\nHun Sen and Ranariddh eventually form a coalition at summit convened by Sihanouk.\nHun Sen remains prime minister, Ranariddh is president of the national assembly, and a new senate will be formed.\nOpposition leader Rainsy left out.\nHe seeks strong assurance of safety should he return to Cambodia.\n",
                    ]

    hypothesis_2 = "China 's government said Thursday that two prominent dissidents arrested this week are suspected of endangering national security _ the clearest sign yet Chinese leaders plan to quash a would-be opposition party .\nOne leader of a suppressed new political party will be tried on Dec. 17 on a charge of colluding with foreign enemies of China '' to incite the subversion of state power , '' according to court documents given to his wife on Monday .\nWith attorneys locked up , harassed or plain scared , two prominent dissidents will defend themselves against charges of subversion Thursday in China 's highest-profile dissident trials in two years .\n"
    references_2 = "Hurricane Mitch, category 5 hurricane, brought widespread death and destruction to Central American.\nEspecially hard hit was Honduras where an estimated 6,076 people lost their lives.\nThe hurricane, which lingered off the coast of Honduras for 3 days before moving off, flooded large areas, destroying crops and property.\nThe U.S. and European Union were joined by Pope John Paul II in a call for money and workers to help the stricken area.\nPresident Clinton sent Tipper Gore, wife of Vice President Gore to the area to deliver much needed supplies to the area, demonstrating U.S. commitment to the recovery of the region.\n"

    all_hypothesis = [hypothesis_1, hypothesis_2]
    all_references = [references_1, references_2]

    scores = evaluator.get_scores(all_hypothesis, all_references)

    for metric, results in sorted(scores.items(), key=lambda x: x[0]):
        if not apply_avg and not apply_best: # value is a type of list as we evaluate each summary vs each reference
            for hypothesis_id, results_per_ref in enumerate(results):
                nb_references = len(results_per_ref['p'])
                for reference_id in range(nb_references):
                    print('\tHypothesis #{} & Reference #{}: '.format(hypothesis_id, reference_id))
                    print('\t' + prepare_results(results_per_ref['p'][reference_id], results_per_ref['r'][reference_id], results_per_ref['f'][reference_id]))
            print()
        else:
            print(prepare_results(results['p'], results['r'], results['f']))
    print()

Evaluation with Avg


TypeError: Rouge.__init__() got an unexpected keyword argument 'length_limit'